# Train a Simplicial Neural Network for Homology Localization (Dist2Cycle)

In this notebook, we will create and train a Simplicial Neural Network for Homology Localization, as proposed in the paper by [Alexandros D. Keros et. al : Dist2Cycle: A Simplicial Neural Network for Homology Localization(2022)](https://ojs.aaai.org/index.php/AAAI/article/view/20673/20432). 

We train the model to perform binary node classification using the KarateClub benchmark dataset. 

The equations of one layer of this neural network are given by:

🟥 $\quad m^{(1 \rightarrow 1)}\_{y \rightarrow x}  = (A \odot (I + L\downarrow)^+{xy}) \cdot h_{y}^{t,(1)}\cdot \Theta^t$

🟧 $\quad m_x^{(1 \rightarrow 1)}  = \sum_{y \in \mathcal{L}\_\downarrow(x)} m_{y \rightarrow x}^{(1 \rightarrow 1)}$

🟩 $\quad m_x^{(1)}  = m^{(1 \rightarrow 1)}_x$

🟦 $\quad h_x^{t+1,(1)} = \sigma(m_{x}^{(1)})$



Where the notations are defined in [Papillon et al : Architectures of Topological Deep Learning: A Survey of Topological Neural Networks (2023)](https://arxiv.org/abs/2304.10031).

In [5]:
import torch
import numpy as np

from toponetx import SimplicialComplex
import toponetx.datasets.graph as graph
import torch_geometric
#from topomodelx.nn.simplicial.hsn_layer import HSNLayer
import os
import torch
#import loaddatas as lds
import torch.nn.functional as F
import numpy as np
import topomodelx.nn.simplicial.bScNet_layer as bScLayer
from sklearn.metrics import roc_auc_score, average_precision_score
from torch.nn.init import xavier_normal_ as xavier
import torch_geometric.transforms as T



In [9]:
#dataset = graph.karate_club(complex_type="simplicial")
#print(dataset)
#Cora Dataset
dataset = torch_geometric.datasets.Planetoid(
    root='tmp/Cora',name='Cora', transform=T.NormalizeFeatures())

data=dataset[0]


In [24]:
#Testing Cora
print(dataset.num_classes)
print(dataset.name)
print(data)
print(data.edge_index[1][1])


7
Cora
Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], train_pos=4488, train_neg=3663498, val_pos=263, val_neg=263, test_pos=527, test_neg=527, total_edges=[3669566, 2], total_edges_y=[3669566])
tensor(2582)


In [11]:
print(data)
model, data = locals()['bScLayer'].call(
    data, dataset.name, data.x.size(1), dataset.num_classes)


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


C:\Users\zia003\OneDrive - CSIRO\CSIRO work\Code\TopoX ICML 2003 challenge\TopoModelX\topomodelx\nn\simplicial\bScNet_layer.py:255: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(g)
C:\Users\zia003\OneDrive - CSIRO\CSIRO work\Code\TopoX ICML 2003 challenge\TopoModelX\topomodelx\nn\simplicial\bScNet_layer.py:340: FutureWarning: incidence_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  B1 = np.array(nx.incidence_matrix(
C:\Users\zia003\OneDrive - CSIRO\CSIRO work\Code\TopoX ICML 2003 challenge\TopoModelX\topomodelx\nn\simplicial\bScNet_layer.py:174: RuntimeWarning: invalid value encountered in matmul
  L0u = B1.T @ B1  # B1 @ D3_n @ B1.T @ inv(D2_1)


In [12]:
def weights_init(m):
    if isinstance(m, torch.nn.Linear):
        xavier(m.weight)
        if not m.bias is None:
            torch.nn.init.constant_(m.bias, 0)


In [14]:
model.apply(weights_init)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0)
best_val_acc = test_acc_same = test_acc_diff = test_acc = 0.0
best_val_roc = test_roc_same = test_roc_diff = test_roc = 0.0
best_val_loss = np.inf
# train and val/test
wait_step = 0


In [16]:
wait_total = 200
total_epochs = 20


def train():
    model.train()
    optimizer.zero_grad()
    emb = model.g_encode(data).clone()
    x, y = model.s_encode(data, emb)  # emb from encode's, i.e., Gconv's output
    loss = F.binary_cross_entropy(x, y)
    loss.backward()
    optimizer.step()
    return x


def test():
    model.eval()
    accs = []
    emb = model.g_encode(data)
    for type in ["val", "test"]:
        pred, y = model.s_encode(data, emb, type=type)
        pred, y = pred.cpu(), y.cpu()
        if type == "val":
            accs.append(F.binary_cross_entropy(pred, y))
            pred = pred.data.numpy()
            roc = roc_auc_score(y, pred)
            accs.append(roc)
            acc = average_precision_score(y, pred)
            accs.append(acc)
        else:
            pred = pred.data.numpy()
            roc = roc_auc_score(y, pred)
            accs.append(roc)
            acc = average_precision_score(y, pred)
            accs.append(acc)
    return accs


def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

# train and test
for epoch in range(1, total_epochs + 1):
    print("epoch is:", epoch)
    pred = train()
    val_loss, val_roc, val_acc, tmp_test_roc, tmp_test_acc = test()
    if val_roc >= best_val_roc:
        test_acc = tmp_test_acc
        test_roc = tmp_test_roc
        best_val_acc = val_acc
        best_val_roc = val_roc
        best_val_loss = val_loss
        wait_step = 0
    else:
        wait_step += 1
        if wait_step == wait_total:
            print('Early stop! Min loss: ', best_val_loss, ', Max accuracy: ', best_val_acc,
                    ', Max roc: ', best_val_roc)
            break
    print(best_val_roc)
#del model
#del data
# print result

#pipeline_acc[Conv_method][data_cnt] = test_acc
#pipeline_roc[Conv_method][data_cnt] = test_roc

#log = 'Epoch: ' + str(
#    total_epochs) + ', dataset name: ' + d_name + ', Method: ' + Conv_method + ' Test pr: {:.4f}, roc: {:.4f} \n'
#print((log.format(pipeline_acc[Conv_method][data_cnt], pipeline_roc[Conv_method][data_cnt])))


epoch is: 1
0.9426332605647039
epoch is: 2
0.9426332605647039
epoch is: 3
0.9426332605647039
epoch is: 4
0.9426332605647039
epoch is: 5
0.9426332605647039
epoch is: 6
0.9426332605647039
epoch is: 7
0.9426332605647039
epoch is: 8
0.9426332605647039
epoch is: 9
0.9426332605647039
epoch is: 10
0.9426332605647039
epoch is: 11
0.9426332605647039
epoch is: 12
0.9426332605647039
epoch is: 13
0.9426332605647039
epoch is: 14
0.9426332605647039
epoch is: 15
0.9426332605647039
epoch is: 16
0.9426332605647039
epoch is: 17
0.9426621752519193
epoch is: 18
0.9426621752519193
epoch is: 19
0.9426621752519193
epoch is: 20
0.9426621752519193
